## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import matplotlib.pyplot as plt 

import requests
import json
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Iskateley,67.6803,53.1512,26.69,96,100,6.96,RU,overcast clouds
1,1,Newport,51.5877,-2.9984,50.00,88,58,7.45,GB,broken clouds
2,2,Clyde River,70.4692,-68.5914,-13.00,77,20,11.50,CA,few clouds
3,3,Mount Gambier,-37.8333,140.7667,78.80,31,65,12.66,AU,broken clouds
4,4,Sabang,5.8933,95.3214,80.94,80,100,27.67,ID,overcast clouds


In [24]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [25]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
3,3,Mount Gambier,-37.8333,140.7667,78.80,31,65,12.66,AU,broken clouds
4,4,Sabang,5.8933,95.3214,80.94,80,100,27.67,ID,overcast clouds
12,12,Belem De Sao Francisco,-8.7539,-38.9658,80.20,61,99,2.19,BR,heavy intensity rain


In [26]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                3
City                   3
Lat                    3
Lng                    3
Max Temp               3
Humidity               3
Cloudiness             3
Wind Speed             3
Country                3
Current Description    3
dtype: int64

In [29]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
3,3,Mount Gambier,-37.8333,140.7667,78.80,31,65,12.66,AU,broken clouds
4,4,Sabang,5.8933,95.3214,80.94,80,100,27.67,ID,overcast clouds
12,12,Belem De Sao Francisco,-8.7539,-38.9658,80.20,61,99,2.19,BR,heavy intensity rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Mount Gambier,AU,78.80,broken clouds,-37.8333,140.7667,
4,Sabang,ID,80.94,overcast clouds,5.8933,95.3214,
12,Belem De Sao Francisco,BR,80.20,heavy intensity rain,-8.7539,-38.9658,


In [30]:
len(hotel_df)

3

In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
    
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")
hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Mount Gambier,AU,78.80,broken clouds,-37.8333,140.7667,
4,Sabang,ID,80.94,overcast clouds,5.8933,95.3214,
12,Belem De Sao Francisco,BR,80.20,heavy intensity rain,-8.7539,-38.9658,Village Hotel


In [22]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Mount Gambier,AU,78.80,broken clouds,-37.8333,140.7667,
4,Sabang,ID,80.94,overcast clouds,5.8933,95.3214,
12,Belem De Sao Francisco,BR,80.20,heavy intensity rain,-8.7539,-38.9658,Village Hotel


In [33]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

FileNotFoundError: [Errno 2] No such file or directory: 'Vacation_Search/WeatherPy_vacation.csv'

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Save the figure.
plt.savefig("Vacation_Search/WeatherPy_vation_map.png")

# 11b. Display the figure
fig

FileNotFoundError: [Errno 2] No such file or directory: 'Vacation_Search/WeatherPy_vation_map.png'

<Figure size 432x288 with 0 Axes>